## IS2_S2_cryo2ice_search.ipynb
Alek Petty, adapted from an earlier M. Bagnardi script.

TRY THE INTERSECTS OPTION!
https://hamedalemo.github.io/advanced-geo-python/lectures/earth_search_tutorial.html
https://stackstac.readthedocs.io/en/v0.3.1/basic.html
https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/#Analyzing-STAC-Metadata


Deescription:
- This notebook searches for semi-coincident ICESat-2 and Sentinel-2 data.
- The code finds Sentinel-2 images with ICESat-2 data falling within the image's footprint.
- The code was updated to read in ATl07 data from the new cloud storage using earthdata, see relevant tutorials:
     https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/notebooks/ICESat-2_Cloud_Access/ATL06-direct-access_rendered.ipynb
     https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/notebooks/ICESat-2_Cloud_Access/nsidc_daac_uwg_cloud_access_tutorial_rendered.ipynb (No longer hosted!!)
- Include new filtering to only look for images where we have cryo2ice (CryoSat-2 coincident) data based on external file. 
 
To do:
- Should we use CLOUD_COVERAGE_ASSESSMENT or CLOUDY_PIXEL_PERCENTAGE?
- Integrate other datasets on STAC. What data is on those compared to the sentinelhub?
- Check all beams instead of just the middle beam.
- Add in other imagery/satellite sensors.

Notes:
- Another way of doing this is to use the RGT (reference ground track) data from ICESat-2.
    This could help if you want to look at future data I guess, but downside is it doesn't say where the data actually is on yhe ground, 
    just theoretical tracks.
- Uses the middle strong beam but could be easily adapted to check across all beams.=
- Tested with the adapted 'geo' conda env on cryocloud if you want to convert into a python script.

Output:

Update history:
 - Current notebook created in Apr 2024.


In [71]:
# Uncomment these out when running for the first time on cryocloud
#%pip install sentinelsat
#%pip install earthdata
#%pip install astropy

In [3]:
# Load modules
import datetime as dt
import h5py
import pandas as pd
# Need to do this for geopandas for some reason
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import glob
from shapely.geometry import Point
import matplotlib.pyplot as plt
import os

import numpy as np
from PIL import Image
import io
import requests
import time
import earthaccess
import xarray as xr
from datetime import datetime
from datetime import timedelta

from pystac_client import Client
#import stackstac

#from astropy.time import Time
#from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt


In [4]:
os.environ["AWS_REQUEST_PAYER"] = "requester"
os.environ["AWS_PROFILE"] = "akpetty"

In [5]:
catalog = Client.open("https://earth-search.aws.element84.com/v1/")

In [6]:
# ICESat-2/earthaccess stuff
auth = earthaccess.login()


Enter your Earthdata Login username:  akpetty
Enter your Earthdata password:  ········


In [8]:
c2ice_df = pd.read_csv('../Data/Cryo2Ice_info_array.csv')
c2ice_df['is2_start_time_dt'] = c2ice_df['start_IS2'].apply(
    lambda x: pd.to_datetime(''.join(x.split(',')), format='%Y %m %d %H %M %S')
)

c2ice_df['is2_end_time_dt'] = c2ice_df['end_IS2'].apply(
    lambda x: pd.to_datetime(''.join(x.split(',')), format='%Y %m %d %H %M %S')
)

#reader.vars.avail()
c2ice_files = c2ice_df["atl07filename"].tolist()

In [ ]:
#find the matched S2 images for each IS2 file
coincident_S2_images=[]

#coincident_S2_mosaics=[]
coincident_S2_image_ids=[]

#number of S2 images for each IS2 granule
number_of_coincident_scenes_in_granule=[]

# coordinate tracks for each coincident IS2 granule
is2_coordinate_tracks_matched=[] 

# save ATL07 info for the coincident S-2 scenes (use middle of the track)
ATL_midtimes_matched=[]
ATL_filenames_matched=[]



# Coarsen by this number of data points to speed things up.
# Can play with this more as desired
NPTS=10
# Time difference in minutes
deltatime=300
# Max cloud cover in the S-2 images
max_cloud_cover=100

# Just do Arctic scenes
lower_lat=40

# Loop over ICESat-2 granules that are in the cryo2ice search
# Adapt this to just use all ATL07 files for a complete (non C2I) search
for c2ice_file in c2ice_files:
    granules_cloud = earthaccess.search_data(
        short_name = 'ATL07',
        version = '006',
        cloud_hosted = True,
        bounding_box = (-180,60,180,90),
        #temporal = ('2020-10-01','2020-10-30'),
        granule_name=c2ice_file,
    )
    print(granules_cloud)
    
    # Open files (not found a way of not needing to do this..)
    atl07_file = earthaccess.open(granules_cloud)
    ATL = h5py.File(atl07_file[0], 'r')
    
    orientation = ATL['/orbit_info/sc_orient'][0]

    # Only use central strong beam locations
    if orientation == 0:
        beamID = 'gt2l'
    elif orientation == 1:
        beamID = 'gt2r'
    else:
        print('Spacecraft orientation not found.')

    # Extract data info from granule
    ATL_start_time = ATL['/ancillary_data/data_start_utc'][0]
    ATL_start = pd.to_datetime(ATL_start_time[:-8].decode('utf-8'), format='%Y-%m-%dT%H:%M:%S')
    ATL_end_time = ATL['/ancillary_data/data_end_utc'][0]
    ATL_end = pd.to_datetime(ATL_end_time[:-8].decode('utf-8'), format='%Y-%m-%dT%H:%M:%S')

    ATL_lon = ATL[beamID + '/sea_ice_segments/longitude'][:]
    ATL_lat = ATL[beamID + '/sea_ice_segments/latitude'][:]

    # Read in the new dist to land variable to filter these values out now we have a lot more land data in rel006!
    ATL_dist_land = ATL[beamID+'/sea_ice_segments/stats/dist2land'][:]
    ATL_lon = ATL_lon[ATL_dist_land>0]
    ATL_lat = ATL_lat[ATL_dist_land>0]
    atl07_coords = list(zip(ATL_lon[::NPTS], ATL_lat[::NPTS])) 

    # Build dataframe with location of data
    ATL_dF = pd.DataFrame({'Longitude': ATL_lon[::NPTS],
                           'Latitude': ATL_lat[::NPTS],
                           })
    ATL_dF['coords'] = list(zip(ATL_dF['Longitude'], ATL_dF['Latitude']))
    ATL_dF['coords'] = ATL_dF['coords'].apply(Point)

    ATL_gfd = gpd.GeoDataFrame(ATL_dF, geometry='coords')
    ATL_gfd = ATL_gfd.set_crs(4326, allow_override=True)

    start_time_window = (ATL_start- dt.timedelta(minutes=deltatime)).strftime('%Y-%m-%d:%H:%M:%SZ')
    end_time_window = (ATL_end+ dt.timedelta(minutes=deltatime)).strftime('%Y-%m-%d:%H:%M:%SZ')

    # Query the S2 STAC catalog for this rough time-period.
    query = catalog.search(
        collections="sentinel-2-l1c",
        #ids='S2B_MSIL1C_20220401T160819_N0400_R140_T30XWR_20220401T181620',
        #ids='S2B_30XWR_20220401_0_L1C',
        #bbox=[-20.7, 64.5, -19.5, 64.8],  # xmin, ymin, xmax, ymax
        datetime=start_time_window+'/'+end_time_window,
        #datetime="2022-04-01:16:08:19Z/2022-04-01:16:08:20Z"
        #datetime="2022-04-01T16:08:18Z/2022-04-01T16:08:20.024Z"
        query={"eo:cloud_cover":{"lt":max_cloud_cover}},
    )
    
    items = list(query.items())
    print(f"Found: {len(items):d} datasets")
    
    # Convert STAC items into a GeoJSON FeatureCollection
    stac_json = query.item_collection_as_dict()
    S2_gdf = gpd.GeoDataFrame.from_features(stac_json, "epsg:4326")
    S2_gdf_subset = S2_gdf[(S2_gdf.bounds.miny > lower_lat)]

    # Key line to see where the ATL07 granule may overlap the S-2 scenes at that time
    points_in_poly = gpd.tools.sjoin(ATL_gfd, S2_gdf_subset)
    #print(points_in_poly)
    
    try:
        # Empty geodataframes threw an exception here
        print('Number of overlapping tiles:', len(points_in_poly['title'].unique()))
    except:
        continue
    


Granules found: 1
[Collection: {'EntryTitle': 'ATLAS/ICESat-2 L3A Sea Ice Height V006'}
Spatial coverage: {'HorizontalSpatialDomain': {'Orbit': {'AscendingCrossing': 48.92460331907133, 'StartLatitude': 27.0, 'StartDirection': 'A', 'EndLatitude': 27.0, 'EndDirection': 'D'}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2020-10-15T01:44:07.383Z', 'EndingDateTime': '2020-10-15T01:50:33.617Z'}}
Size(MB): 181.3192491531372
Data: ['https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL07/006/2020/10/15/ATL07-01_20201015012229_03190901_006_01.h5']]
Opening 1 granules, approx size: 0.18 GB
using provider: NSIDC_CPRD


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Found: 20851 datasets
Granules found: 1
[Collection: {'EntryTitle': 'ATLAS/ICESat-2 L3A Sea Ice Height V006'}
Spatial coverage: {'HorizontalSpatialDomain': {'Orbit': {'AscendingCrossing': -48.924131734392304, 'StartLatitude': 27.0, 'StartDirection': 'A', 'EndLatitude': 27.0, 'EndDirection': 'D'}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2021-04-02T00:03:38.476Z', 'EndingDateTime': '2021-04-02T00:18:35.195Z'}}
Size(MB): 376.94387912750244
Data: ['https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL07/006/2021/04/02/ATL07-01_20210401235002_01251101_006_02.h5']]
Opening 1 granules, approx size: 0.37 GB
using provider: NSIDC_CPRD


...

Found: 14042 datasets
Granules found: 1
[Collection: {'EntryTitle': 'ATLAS/ICESat-2 L3A Sea Ice Height V006'}
Spatial coverage: {'HorizontalSpatialDomain': {'Orbit': {'AscendingCrossing': 88.11856585176837, 'StartLatitude': 27.0, 'StartDirection': 'A', 'EndLatitude': 27.0, 'EndDirection': 'D'}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2021-04-25T13:52:09.924Z', 'EndingDateTime': '2021-04-25T14:03:37.452Z'}}
Size(MB): 204.7309274673462
Data: ['https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL07/006/2021/04/25/ATL07-01_20210425133419_04851101_006_02.h5']]
Opening 1 granules, approx size: 0.2 GB
using provider: NSIDC_CPRD


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Found: 14381 datasets


In [27]:
ATL_gfd.bounds

,minx,miny,maxx,maxy
0,28.079709,82.551538,28.079709,82.551538
1,28.070940,82.555476,28.070940,82.555476
2,28.060864,82.560020,28.060864,82.560020
3,28.048393,82.565629,28.048393,82.565629
4,28.039280,82.569732,28.039280,82.569732
...,...,...,...,...
14049,-131.649014,72.788306,-131.649014,72.788306
14050,-131.654817,72.775801,-131.654817,72.775801
14051,-131.667712,72.748034,-131.667712,72.748034
14052,-131.710863,72.654627,-131.710863,72.654627


In [25]:
ATL_start = start_time_window.strftime('%Y-%m-%d:%H:%M:%SZ')
ATL_start

'2020-10-14:20:44:07Z'

In [94]:
coincident_S2_images

[]

In [ ]:
S2[scene_number].toList(number_of_scenes[scene_number]).getInfo()[i]['properties']